In [1]:
# Libraries
# ==============================================================================
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
%config Completer.use_jedi = False

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skforecast.utils import check_y
from skforecast.utils import check_exog
from skforecast.utils import preprocess_y
from skforecast.utils import preprocess_last_window
from skforecast.utils import preprocess_exog
from skforecast.utils import expand_index
from skforecast.utils import check_predict_input

In [50]:
import joblib

def save_forecaster(forecaster, file_name: str, verbose: bool=True) -> None:
    '''
    Save forecaster model using joblib.

    Parameters
    ----------
    forecaster: forecaster object from skforecast library.
        Model created with skforecast library.

    file_name: str
        File name given to the object.
        
    verbose: bool, default True
        Print info about the forecaster saved

    Returns 
    -------
    None
    '''

    joblib.dump(forecaster, filename=file_name)

    if verbose:
        forecaster.summary()


def load_forecaster(file_name: str, verbose: bool=True):
    '''
    Save forecaster model using joblib.

    Parameters
    ----------
    forecaster: forecaster object from skforecast library.
        Forecaster created with skforecast library.

    file_name: str
        File name given to the object.

    verbose: bool, default True
        Print summary about the forecaster loaded.

    Returns 
    -------
    Forecaster
        Forecaster created with skforecast library.
    '''

    forecaster = joblib.load(filename=file_name)

    if verbose:
        forecaster.summary()

    return forecaster


In [61]:
joblib.hash(forecaster.regressor) == joblib.hash(forecaster_loaded.regressor)

True

In [72]:
import sklearn


issubclass(type(forecaster.regressor), sklearn.base.BaseEstimator)

True

In [60]:
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
forecaster = ForecasterAutoreg(regressor=RandomForestRegressor(), lags=3)
save_forecaster(forecaster=forecaster, file_name='forecaster.py', verbose=False)
forecaster_loaded = load_forecaster(file_name='forecaster.py', verbose=False)

os.de

In [30]:
from pickle import dumps
serialization_1 = dumps(forecaster)
serialization_2 = dumps(forecaster_loaded)

serialization_1 == serialization_2

False

In [83]:
(forecaster.last_window.values == forecaster.last_window.values).all()

True

In [91]:
import os
import joblib
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor


def test_save_and_load_forecaster_persistence():
    ''' 
    Test if a loaded forecaster is exactly the same as the original one.
    '''

    forecaster = ForecasterAutoreg(regressor=LinearRegression(), lags=3)
    rng = np.random.default_rng(12345)
    y = pd.Series(rng.normal(size=100))
    forecaster.fit(y=y)
    save_forecaster(forecaster=forecaster, file_name='forecaster.py', verbose=False)
    forecaster_loaded = load_forecaster(file_name='forecaster.py', verbose=False)
    os.remove('forecaster.py')

    for key in vars(forecaster).keys():
        attribute_forecaster = forecaster.__getattribute__(key)
        attribute_forecaster_loaded = forecaster_loaded.__getattribute__(key)

        if key == 'regressor':
            assert joblib.hash(attribute_forecaster) == joblib.hash(attribute_forecaster_loaded)
        
        elif isinstance(attribute_forecaster, (np.ndarray, pd.Series, pd.DataFrame, pd.Index)):
            assert (attribute_forecaster == attribute_forecaster_loaded).all()
        else:
            assert attribute_forecaster == attribute_forecaster_loaded
    

In [6]:
# Unit test check_input_predict
# ==============================================================================
import pytest
from pytest import approx
import numpy as np
import pandas as pd
from skforecast.utils import check_predict_input

def test_check_input_predict_exception_when_fitted_is_False():
    '''
    Test exception is raised when fitted is False.
    '''
    with pytest.raises(Exception):
        check_predict_input(
            steps          = 5,
            fitted         = False,
            included_exog  = False,
            index_type     = None,
            index_freq     = None,
            window_size    = None,
            last_window    = None,
            exog           = None,
            exog_type      = None,
            exog_col_names = None,
            max_steps      = None,
        )
        
        
def test_check_input_predict_exception_when_steps_is_lower_than_1():
    '''
    Test exception is steps is a value lower than 1.
    '''
    with pytest.raises(Exception):
        check_predict_input(
            steps          = -5,
            fitted         = True,
            included_exog  = False,
            index_type     = None,
            index_freq     = None,
            window_size    = None,
            last_window    = None,
            exog           = None,
            exog_type      = None,
            exog_col_names = None,
            max_steps      = None,
        )
        
def test_check_input_predict_exception_when_steps_is_greater_than_max_steps():
    '''
    '''
    with pytest.raises(Exception):
        check_predict_input(
            steps          = 20,
            fitted         = True,
            included_exog  = False,
            index_type     = None,
            index_freq     = None,
            window_size    = None,
            last_window    = None,
            exog           = None,
            exog_type      = None,
            exog_col_names = None,
            max_steps      = 10,
        )
        
def test_check_input_predict_exception_when_exog_is_not_none_and_included_exog_is_false():
    '''
    '''
    with pytest.raises(Exception):
        check_predict_input(
            steps          = 5,
            fitted         = True,
            included_exog  = False,
            index_type     = None,
            index_freq     = None,
            window_size    = None,
            last_window    = None,
            exog           = np.arange(10),
            exog_type      = None,
            exog_col_names = None,
            max_steps      = None,
        )
        
def test_check_input_predict_exception_when_exog_is_none_and_included_exog_is_true():
    '''
    '''
    with pytest.raises(Exception):
        check_predict_input(
            steps          = 5,
            fitted         = True,
            included_exog  = True,
            index_type     = None,
            index_freq     = None,
            window_size    = None,
            last_window    = None,
            exog           = None,
            exog_type      = None,
            exog_col_names = None,
            max_steps      = None,
        )
        
def test_check_input_predict_exception_when_len_exog_is_less_than_steps():
    '''
    '''
    with pytest.raises(Exception):
        check_predict_input(
            steps          = 10,
            fitted         = True,
            included_exog  = True,
            index_type     = None,
            index_freq     = None,
            window_size    = None,
            last_window    = None,
            exog           = np.arange(5),
            exog_type      = None,
            exog_col_names = None,
            max_steps      = None,
        )
        
def test_check_input_predict_exception_when_exog_is_not_padas_series_or_dataframe():
    '''
    '''
    with pytest.raises(Exception):
        check_predict_input(
            steps          = 10,
            fitted         = True,
            included_exog  = True,
            index_type     = None,
            index_freq     = None,
            window_size    = 5,
            last_window    = None,
            exog           = np.arange(10),
            exog_type      = None,
            exog_col_names = None,
            max_steps      = None,
        )
        
def test_check_input_predict_exception_when_exog_has_missing_values():
    '''
    '''
    with pytest.raises(Exception):
        check_predict_input(
            steps          = 10,
            fitted         = True,
            included_exog  = True,
            index_type     = None,
            index_freq     = None,
            window_size    = 5,
            last_window    = None,
            exog           = pd.Series([1, 2, 3, np.nan]),
            exog_type      = None,
            exog_col_names = None,
            max_steps      = None,
        )
        
def test_check_input_predict_exception_when_exog_is_not_of_exog_type():
    '''
    '''
    with pytest.raises(Exception):
        check_predict_input(
            steps          = 10,
            fitted         = True,
            included_exog  = True,
            index_type     = None,
            index_freq     = None,
            window_size    = None,
            last_window    = None,
            exog           = np.arange(10),
            exog_type      = pd.Series,
            exog_col_names = None,
            max_steps      = None,
        )
        
def test_check_input_predict_exception_when_exog_is_dataframe_without_columns_in_exog_col_names():
    '''
    '''
    with pytest.raises(Exception):
        check_predict_input(
            steps          = 2,
            fitted         = True,
            included_exog  = True,
            index_type     = None,
            index_freq     = None,
            window_size    = None,
            last_window    = None,
            exog           = pd.DataFrame(np.arange(10).reshape(5,2), columns=['col1', 'col2']),
            exog_type      = pd.DataFrame,
            exog_col_names = ['col1', 'col3'],
            max_steps      = None,
        )
        
        
def test_check_input_predict_exception_when_exog_index_is_not_of_index_type():
    '''
    '''
    with pytest.raises(Exception):
        check_predict_input(
            steps          = 10,
            fitted         = True,
            included_exog  = True,
            index_type     = pd.DatetimeIndex,
            index_freq     = None,
            window_size    = None,
            last_window    = None,
            exog           = pd.Series(np.arange(10)),
            exog_type      = pd.Series,
            exog_col_names = None,
            max_steps      = None,
        )
        
def test_check_input_predict_exception_when_exog_index_frequency_is_not_index_freq():
    '''
    '''
    with pytest.raises(Exception):
        check_predict_input(
            steps          = 10,
            fitted         = True,
            included_exog  = True,
            index_type     = pd.DatetimeIndex,
            index_freq     = 'Y',
            window_size    = None,
            last_window    = None,
            exog           = pd.Series(np.arange(10), index=pd.date_range(start='1/1/2018', periods=10)),
            exog_type      = pd.Series,
            exog_col_names = None,
            max_steps      = None,
        )
        
def test_check_input_predict_exception_when_length_last_window_is_lower_than_window_size():
    '''
    '''
    with pytest.raises(Exception):
        check_predict_input(
            steps          = 10,
            fitted         = True,
            included_exog  = True,
            index_type     = pd.RangeIndex,
            index_freq     = None,
            window_size    = 10,
            last_window    = pd.Series(np.arange(5)),
            exog           = pd.Series(np.arange(10)),
            exog_type      = pd.Series,
            exog_col_names = None,
            max_steps      = None,
        )
        
        
def test_check_input_predict_exception_when_last_window_is_not_padas_series():
    '''
    '''
    with pytest.raises(Exception):
        check_predict_input(
            steps          = 10,
            fitted         = True,
            included_exog  = True,
            index_type     = pd.RangeIndex,
            index_freq     = None,
            window_size    = 5,
            last_window    = np.arange(5),
            exog           = pd.Series(np.arange(10)),
            exog_type      = pd.Series,
            exog_col_names = None,
            max_steps      = None,
        )
        
        
def test_check_input_predict_exception_when_last_window_has_missing_values():
    '''
    '''
    with pytest.raises(Exception):
        check_predict_input(
            steps          = 10,
            fitted         = True,
            included_exog  = True,
            index_type     = pd.RangeIndex,
            index_freq     = None,
            window_size    = 5,
            last_window    = pd.Series([1, 2, 3, 4, 5, np.nan]),
            exog           = pd.Series(np.arange(10)),
            exog_type      = pd.Series,
            exog_col_names = None,
            max_steps      = None,
        )

In [8]:
test_check_input_predict_exception_when_fitted_is_False()
test_check_input_predict_exception_when_steps_is_lower_than_1()
test_check_input_predict_exception_when_steps_is_greater_than_max_steps()
test_check_input_predict_exception_when_exog_is_not_none_and_included_exog_is_false()
test_check_input_predict_exception_when_exog_is_none_and_included_exog_is_true()
test_check_input_predict_exception_when_exog_is_not_padas_series_or_dataframe()
test_check_input_predict_exception_when_exog_has_missing_values()
test_check_input_predict_exception_when_len_exog_is_less_than_steps()
test_check_input_predict_exception_when_exog_is_not_of_exog_type()
test_check_input_predict_exception_when_exog_is_dataframe_without_columns_in_exog_col_names()
test_check_input_predict_exception_when_exog_index_is_not_of_index_type()
test_check_input_predict_exception_when_exog_index_frequency_is_not_index_freq()
test_check_input_predict_exception_when_length_last_window_is_lower_than_window_size()
test_check_input_predict_exception_when_last_window_is_not_padas_series()
test_check_input_predict_exception_when_last_window_has_missing_values()

In [19]:
import pytest
import numpy as np
import pandas as pd
from skforecast.utils import exog_to_multi_output

def test_exog_to_multi_output_when_lags_3_steps_2_exog_numpy_1d():
    '''
    Test exog_to_multi_output results when using lags 3, steps 2 and exog is a  
    1d numpy array.
    '''
    exog = np.arange(10)
    results = exog_to_multi_output(exog=exog, steps=2)
    expected = np.array([[0, 1],
                        [1, 2],
                        [2, 3],
                        [3, 4],
                        [4, 5],
                        [5, 6],
                        [6, 7],
                        [7, 8],
                        [8, 9]])

    assert results == approx(expected)



def test_exog_to_multi_output_when_lags_3_steps_2_exog_numpy_array_2d():
    '''
    Test exog_to_multi_output results when using lags 3, steps 2 and exog is a  
    2d numpy array.
    '''
    exog = np.column_stack([np.arange(100, 110), np.arange(1000, 1010)])
    results = exog_to_multi_output(exog=exog, steps=2)
    expected = np.array([[ 100,  101, 1000, 1001],
                        [ 101,  102, 1001, 1002],
                        [ 102,  103, 1002, 1003],
                        [ 103,  104, 1003, 1004],
                        [ 104,  105, 1004, 1005],
                        [ 105,  106, 1005, 1006],
                        [ 106,  107, 1006, 1007],
                        [ 107,  108, 1007, 1008],
                        [ 108,  109, 1008, 1009]])

    assert results == approx(expected)



def test_exog_to_multi_output_when_lags_2_steps_3_exog_numpy_array_2d():
    '''
    Test exog_to_multi_output results when using lags 2, steps 3 and exog is a  
    2d numpy array.
    '''
    exog = np.column_stack([np.arange(100, 110), np.arange(1000, 1010)])
    results = exog_to_multi_output(exog=exog, steps=3)
    expected = np.array([[ 100,  101,  102, 1000, 1001, 1002],
                        [ 101,  102,  103, 1001, 1002, 1003],
                        [ 102,  103,  104, 1002, 1003, 1004],
                        [ 103,  104,  105, 1003, 1004, 1005],
                        [ 104,  105,  106, 1004, 1005, 1006],
                        [ 105,  106,  107, 1005, 1006, 1007],
                        [ 106,  107,  108, 1006, 1007, 1008],
                        [ 107,  108,  109, 1007, 1008, 1009]])

    assert results == approx(expected)

In [20]:
test_exog_to_multi_output_when_lags_3_steps_2_exog_numpy_1d()
test_exog_to_multi_output_when_lags_3_steps_2_exog_numpy_array_2d()
test_exog_to_multi_output_when_lags_2_steps_3_exog_numpy_array_2d()